In [1]:
%pylab inline
from ipywidgets import interact
import numpy.linalg as la
sx,sy,sz,s0 = np.array([[0, 1], [1, 0]], dtype=complex) \
    ,np.array([[0, -1.j], [1.j, 0]], dtype=complex)  \
    ,np.array([[1, 0] ,   [0,  -1]], dtype=complex)  \
    ,np.array([[1, 0] ,   [0,   1]], dtype=complex)

Populating the interactive namespace from numpy and matplotlib


In [3]:
def p_ip_spectrum(NN=6, mu=1, delta=1., d_mu = 0.2 \
            , print_errors = 1, C=1, d_C = 0, rseed=123):
    """ 
    Calculate eigenvalues and eigenstates of QWZ in real space.  
    Shape of simulated area can be arbitrary.
    Include disorder in u and c (u and mu are used interchangeably throughout)

    """
    random.seed(rseed) #for reproducability
    
    Nx, Ny = 4*NN, 4*NN
    y, x = meshgrid(range(Ny), range(Nx))
    x, y = x.flatten(), y.flatten() # generate coordinates

    delta += 0.j
    tx, ty = 1., 1.
    mu1 = mu + d_mu*random.randn(Nx*Ny) #introduce disorder into mu/u

    #####################################
    # Prepare shape of sample, example: circle:
    x0, y0 = Nx//2, Ny//2
    is_inside = ( ((x-Nx/2)**2 + (y-Ny/2)**2 < (Nx//2-0.1)**2) )
    
    # plot simulated area:
    plotit = False
    
    if plotit:
        plot(x, y, 'o', markeredgecolor='green', markerfacecolor='w', markersize=6)
        plot(x[is_inside], y[is_inside], 'b+')
        axis('equal');
        xlabel('x')
        ylabel('y');
        show()

    # define some helper matrices to be used in Hamiltonian construction:
    idx, idy = eye(Nx), eye(Ny)
    odx, ody = roll(idx, 1, axis=0), roll(idy, 1, axis=0)
    
    # 1st construct up->up matrix element: nearest neighbor hopping + onsite chemical potential:
    h0 = kron(idx, ody) + kron(odx, idy)
   
    h0 += h0.T.conj()
    h0 += diag(mu1)  #onsite chemical potential, mu is u
    #print("shape of h0: ", shape(h0))
    
    # construct down->up matrix element: hopping
    dd = delta * (1.j*kron(odx, idy) + kron(idx, ody))  
    dd -= dd.T  # because of the i's in the definition
    #print("dd: ", shape(dd))
    
    #construct top right coupling matrix for two QWZ layers
    C1 = C + d_C*random.randn(Nx*Ny)

    cc00 = C1*np.zeros([Nx*Ny, Ny*Nx])
    cc01 = -C1*eye(Nx*Ny, Ny*Nx)
    cc10 = C1*eye(Nx*Ny, Nx*Ny)
    cc11 = C1*np.zeros([Nx*Ny, Ny*Nx])
       
    # cut region of interest: keep only those that hop into,
    #   or out of the region of interest
    h0in = h0[:, is_inside][is_inside, :]
    ddin = dd[:, is_inside][is_inside, :]
        
    cc00in = cc00[:, is_inside][is_inside, :]
    cc10in = cc10[:, is_inside][is_inside, :]
    cc11in = cc11[:, is_inside][is_inside, :]
    cc01in = cc01[:, is_inside][is_inside, :]
       
        
    # construct the Hamiltonian from these blocks  (follow general BdG recipe): 
    HH = concatenate((concatenate((h0in,         ddin),         axis=1), 
                      concatenate((ddin.conj().T, -h0in.conj()), axis=1)), axis=0)
    
    
    CC = concatenate((concatenate((cc00in,         cc01in),         axis=1), 
                      concatenate((cc10in, cc11in), axis=1)), axis=0)
    #Build up the full hamiltonian:
    HH4 = concatenate((concatenate((HH,     CC),         axis=1), 
                      concatenate((CC.T.conj(), np.conj(HH)), axis=1)), axis=0)
    
    #print("shape of HH: ", shape(HH))
    N = shape(HH)[0]//2
    SX = np.kron(sx, np.eye(N))
    
    eee, vvv = eigh(HH4)

    return x[is_inside], y[is_inside], eee, vvv #, la.det(UU)



In [4]:
'''
main function that calls p_ip_spectrum
'''
def play(ee=0, zero_energy = 0.05, d_mu=0., d_C = 0., C=1, u=0, rseed=124):
    
    x1, y1, eee, vvv = p_ip_spectrum(NN=4, mu=u, delta=1., d_mu = d_mu, \
                                     rseed=rseed, print_errors = 1, d_C = d_C, C=C)
    
    N = len(eee)//2
    size_pref = len(eee) #basis size for plotting edge localization degree
    show()
    fig,ax = subplots(1, 2, figsize = (12, 6) \
                     , gridspec_kw={'width_ratios': [1, 3]} )
    ax[0].plot(eee, "k.")
    

    print(shape(eee))
    print(shape(vvv))
    
    print(shape(eee))
    eps=1e-15
       
    
    
    wA = np.zeros(N//2) #weight of edge states in position space
    wB = np.zeros_like(wA)
    
    #calculate and print some defining energy levels
    tenth_energy = np.partition(abs(eee), 9)[9]
    twentieth_energy = np.partition(abs(eee), 19)[19] #twentieth smallerst energy
    fifty_energy = np.partition(abs(eee), 49)[49]
    print("ten smallest:", tenth_energy)
    print("twenty smallest:", twentieth_energy)
    print("fifty smallest:", fifty_energy)
    largest_energy = np.partition(abs(eee), len(eee)-1)[len(eee)-1] #largest energy
    print("largest: ", largest_energy)
    
    
    totalA = np.zeros([len(x1), len(y1)])
    
    #calculate support on position coordinates for zero energy states
    for ii in range(len(eee)):
        if abs(eee[ii]-ee)<zero_energy: #check zero energy; by zero we mean smaller than zero_energy
            #can also modify ee to replace centering on 0 energy with centering on ee with radius still zero_energy
            wA += 1/2*np.abs(vvv[:N//2, ii]-1.j*vvv[-N//2:, ii])**2 + 1/2*np.abs(vvv[N//2:N, ii]-1.j*vvv[N:(3*N)//2:, ii])**2
            wB += 1/2*np.abs(vvv[:N//2, ii]+1.j*vvv[-N//2:, ii])**2 + 1/2*np.abs(vvv[N//2:N, ii]+1.j*vvv[N:(3*N)//2:, ii])**2
            ax[0].plot(ii, eee[ii], "ro")
    #col_obj sets the colour at the given real-space site and is proportional to the difference between support on either site
    #this means that a color corresponding to zero means equal support on both sublattices (sublattice here means whether the
    #eigenvalue of the chiral operator is +1 or -1) 
    col_obj = (wA-wB)/(wA+wB+eps)
    map_wf = ax[1].scatter(x1, y1, s=size_pref*(wA + wB), vmin=-1, vmax=1, c = col_obj, marker='s' )
    #ax.matshow()
    colorbar(map_wf, ax=ax[1])
    ax[1].axis('equal')
    ax[1].set_xlabel('x')
    ax[1].set_ylabel('y')
    plt.savefig("realspace bhz.pdf")
    show()
   

    
interact(play, ee=(-2, 2, 0.1) , zero_energy=(0, 2, 0.01), d_mu = (0, 2, 0.01), \
         rseed=(0,200,1), C= (0, 1, 0.01), u =(-2.5, 2.5, 0.01))


interactive(children=(FloatSlider(value=0.0, description='ee', max=2.0, min=-2.0), FloatSlider(value=0.05, des…

<function __main__.play(ee=0, zero_energy=0.05, d_mu=0.0, d_C=0.0, C=1, u=0, rseed=124)>